In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
aisles = pd.read_csv("../input/instacart-market-basket-analysis/aisles.csv")
departments = pd.read_csv("../input/instacart-market-basket-analysis/departments.csv")
order_products_train = pd.read_csv("../input/instacart-market-basket-analysis/order_products__prior.csv")
order_products_test = pd.read_csv("../input/instacart-market-basket-analysis/order_products__train.csv")
orders = pd.read_csv("../input/instacart-market-basket-analysis/orders.csv")
products = pd.read_csv("../input/instacart-market-basket-analysis/products.csv")

In [ ]:
orders.fillna(0,inplace=True)

In [ ]:
train_df1 = order_products_train.groupby("order_id")["add_to_cart_order"].max().reset_index()
train_df2 = order_products_train.groupby("order_id")["reordered"].sum().reset_index()
train_df2.drop("order_id",axis =1,inplace = True)
train_df1.head()
train_df2.head()

In [ ]:
frames = [train_df1,train_df2]
train_df = pd.concat(frames,axis = 1)
train_df

In [ ]:
test_df1 = order_products_test.groupby("order_id")["add_to_cart_order"].max().reset_index()
test_df2 = order_products_test.groupby("order_id")["reordered"].sum().reset_index()
test_df3 = order_products_test.groupby("order_id")["product_id"].unique().reset_index()
test_df2.drop("order_id",axis =1,inplace = True)
test_df1.head()
test_df2.head()

In [ ]:
frames = [test_df1,test_df2]
test_df = pd.concat(frames,axis = 1)
test_df

In [ ]:
frames = [train_df,test_df]
grouped_df1 = pd.concat(frames, axis = 0)
grouped_df1

In [ ]:
order = orders[["order_id","user_id","order_number","days_since_prior_order","eval_set"]]
order

In [ ]:
order.drop(order.loc[order['eval_set']=='test'].index, inplace=True)
order

In [ ]:
orders1= order.groupby(['user_id']).sum()
orders1.head()

In [ ]:
orders2 = order.groupby(['user_id']).count()
orders2 = orders2.drop("eval_set",axis = 1)
orders2.head()

In [ ]:
data1 = pd.DataFrame(orders1/(orders2))
data1 = data1.drop(["order_id","order_number"],axis = 1)
data1.head()

In [ ]:
data2 = pd.DataFrame(orders2['order_number'])
data2.head()

In [ ]:
frames = [data1,data2]
grouped_df2 = pd.concat(frames,axis =1)
grouped_df2.head()

In [ ]:
grouped_df2.columns = ['average_days_since_prior_order','number_of_visits']
grouped_df2.reset_index(level=0, inplace=True)
grouped_df2

In [ ]:
visits = grouped_df2.drop(["user_id","average_days_since_prior_order"],axis = 1)
visits

In [ ]:
grouped_df1

In [ ]:
x = order.drop(["order_number","days_since_prior_order","eval_set"],axis = 1)


In [ ]:
merge1 = pd.merge(left = grouped_df1, right = x, how = "left", on = "order_id")
merge1

In [ ]:
merge2 = pd.merge(left = merge1 ,right = grouped_df2, how = "left", on = "user_id")
merge2

In [ ]:
merge2.sort_values("user_id")

In [ ]:
train_table = merge2.groupby("user_id").sum()
train_table = train_table.drop(["order_id","number_of_visits"],axis = 1)
train_table.reset_index(level=0, inplace=True)
train_table

In [ ]:
frames = [train_table,visits]
train_table = pd.concat(frames,axis = 1)
train_table

**MODEL**

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

In [ ]:
X = train_table.iloc[:,[1,4]].values


In [ ]:
wcss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters=i, init='k-means++',  max_iter=300, n_init=10,
                   random_state=0)
    kmeans.fit(X)
    inertia = kmeans.inertia_
    wcss.append(inertia)
    print('For n_cluster =', i, 'The inertia is:', inertia)


In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(range(1, 11), wcss,marker='o',color='red')
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
algorithm = (KMeans(n_clusters = 4 ,init='k-means++', n_init = 10 ,max_iter=300, 
                        tol=0.0001,  random_state= 111  , algorithm='elkan') )
algorithm.fit(X)
labels1 = algorithm.labels_
centroids1 = algorithm.cluster_centers_

In [ ]:
kmeans = KMeans(n_clusters = 6, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(X)
np.unique(y_kmeans)

In [ ]:
kmeans.cluster_centers_

In [ ]:
# Visualising the clusters
plt.figure(figsize=(15,7))
sns.scatterplot(X[y_kmeans == 0, 0], X[y_kmeans == 0, 1], color = 'aqua', label = 'Cluster 1',s=50)
sns.scatterplot(X[y_kmeans == 1, 0], X[y_kmeans == 1, 1], color = 'blue', label = 'Cluster 2',s=50)
sns.scatterplot(X[y_kmeans == 2, 0], X[y_kmeans == 2, 1], color = 'green', label = 'Cluster 3',s=50)
sns.scatterplot(X[y_kmeans == 3, 0], X[y_kmeans == 3, 1], color = 'grey', label = 'Cluster 4',s=50)
sns.scatterplot(X[y_kmeans == 4, 0], X[y_kmeans == 4, 1], color = 'orange', label = 'Cluster 5',s=50)
sns.scatterplot(X[y_kmeans == 5, 0], X[y_kmeans == 5, 1], color = 'pink', label = 'Cluster 6',s=50)
sns.scatterplot(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], color = 'red', 
                label = 'Centroids',s=300,marker=',')
plt.grid(False)
plt.title('Clusters of customers')
plt.show()